hi I am **markdown**

In [1]:
### Geo data ##################################

import json

with open('data/plz-ZH-lowres.geojson') as f:
    geo_data = json.load(f)

# copy plz to be used as key for choro_data
for feature in geo_data['features']:
    feature['plz'] = feature['properties']['plz']

In [2]:
### Covid data ################################

import pandas
import collections
covid_all = pandas.read_excel('data/f3.xlsx')

def mk_choro_data(d):
    covid_now = covid_all[covid_all['Date'] == str(d)]
    # display(covid_now)
    return collections.defaultdict(
        lambda: 0,
        zip(covid_now['PLZ'], covid_now['rate7_100K'])
    )

In [3]:
### Date slider ###############################

import ipywidgets as widgets
import pandas as pd
from datetime import date, timedelta

# TODO when I grow up, I should show the averaging interval, and have a scale underneath

start_date = min(covid_all['Date']).date()  # TODO + 7 or what?
end_date   = max(covid_all['Date']).date()
dates      = pd.date_range(start_date, end_date, freq='D')

def show_date(date):
    fmt = '%b %d'
    interval_start = date - timedelta(days=7)
    return '{} - {}'.format(interval_start.strftime(fmt), date.strftime(fmt))

options = [(show_date(date), date) for date in dates]

date_slider = widgets.SelectionSlider(
    options=options,
    value=end_date,
    description='Date:',
    orientation='horizontal',
    continuous_update=False,
    layout={'width': '400px'},
)

In [4]:
# from sidecar import Sidecar
# sc = Sidecar(title='map')

In [5]:
### Map #######################################

import ipyleaflet


### TODO move these:

# empty_promises = widgets.HTML('I promise to show a graph here one day!')

info = widgets.HTML('Hover over an area')
# info.layout.width  = '200px'
info.layout.margin = '5px 10px'


m = ipyleaflet.Map(center=(47.36667, 8.55), zoom=12, zoom_control=False, scroll_wheel_zoom=True)
m.layout.height = '600px'

m.add_control(ipyleaflet.WidgetControl(widget=date_slider, position='bottomleft'))
m.add_control(ipyleaflet.FullScreenControl(position='topright'))
m.add_control(ipyleaflet.ZoomControl(position='topright'))
m.add_control(ipyleaflet.WidgetControl(widget=info, position='topleft'))


# with sc:
#     display(m)
#     sc.clear_output(wait=True)
display(m)

Map(center=[47.36667, 8.55], controls=(AttributionControl(options=['position', 'prefix'], position='bottomrigh…

In [6]:
from branca.colormap import linear

covid_layer = ipyleaflet.Choropleth(
    name="Covid19 some data ahem TODO",
    geo_data=geo_data,
    choro_data=mk_choro_data(date_slider.value),
    key_on='plz',
    value_min=-30,
    value_max=100,
    colormap=linear.Purples_07,
    style={'fillOpacity': 0.6, 'weight': 2, 'opacity': 0, 'color': '#529'},
    hover_style={'opacity': 1}
)
m.add_layer(covid_layer)

def on_date_change(change):
    display(change)
    covid_layer.choro_data = mk_choro_data(change['new'])
    
date_slider.observe(on_date_change, names='value')
# date_slider.observe(on_date_change)

def update_info(feature, **kwargs):
    plz = feature['properties']['plz']
    info.value = f'''
        <span style="font-size: 1.6em">{plz} {feature['properties']['ortbez']}</span><br/>
        <b style="font-size: 2.2em">{covid_layer.choro_data[plz]:.2f}</b><br/>
        cases/week/100k
    '''
    
covid_layer.on_hover(update_info)